<a href="https://colab.research.google.com/github/dots13/ForecastingStickerSalesKaggle/blob/main/no_model_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import requests

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
# Function to fetch GDP per capita for a given country and year
def get_gdp_per_capita(alpha3, year):
    """
    Fetch GDP per capita for a specific country and year from the World Bank API.

    """
    url = f'https://api.worldbank.org/v2/country/{alpha3}/indicator/NY.GDP.PCAP.CD?date={year}&format=json'
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data[1][0]['value'] if data[1] else None
    except (requests.RequestException, KeyError, IndexError) as e:
        print(f"Error fetching data for {alpha3} in {year}: {e}")
        return None

In [ ]:
def create_gdp_dataframe(alpha3s, years, country_names):
    """
    Create a DataFrame of normalized GDP per capita ratios for multiple countries and years.

    """
    # Fetch GDP data for all countries and years
    gdp_data = [
        [get_gdp_per_capita(alpha3, year) for year in years]
        for alpha3 in alpha3s
    ]

    # Create a DataFrame with countries as rows and years as columns
    gdp_df = pd.DataFrame(gdp_data, index=country_names, columns=years)

    # Normalize GDP values by dividing by the column sum (yearly total)
    gdp_df = gdp_df / gdp_df.sum(axis=0)

    # Reshape the DataFrame into long format
    gdp_df = gdp_df.reset_index().rename(columns={'index': 'country'})
    gdp_df = gdp_df.melt(id_vars=['country'], var_name='year', value_name='ratio')

    return gdp_df

In [ ]:
def adjust_ratios(gdp_df, adjustments):
    """
    Adjust GDP ratios for specific countries based on custom rules.

    Parameters:
    - gdp_df: DataFrame containing GDP ratios.
    - adjustments: Dictionary with country names as keys and adjustment values.

    Returns:
    - Adjusted DataFrame with updated ratios.
    """
    adjusted_df = gdp_df.copy()

    # Apply adjustments safely
    for country, adjustment in adjustments.items():
        if country in adjusted_df['country'].unique():
            adjusted_df.loc[adjusted_df['country'] == country, 'ratio'] = (
                adjusted_df.loc[adjusted_df['country'] == country, 'ratio'] - adjustment
            ).clip(lower=0)  # Ensure ratios don't become negative

    return adjusted_df

In [ ]:
alpha3s = ['CAN', 'FIN', 'ITA', 'KEN', 'NOR', 'SGP']
years = range(2010, 2020)
country_names = np.array(['Canada', 'Finland', 'Italy', 'Kenya', 'Norway', 'Singapore'])  # Sorted automatically
gdp_ratios_df = create_gdp_dataframe(alpha3s, years, country_names)
adjustments = {'Kenya': 0.0007}
gdp_per_capita_filtered_ratios_df = adjust_ratios(gdp_ratios_df, adjustments)
print(gdp_per_capita_filtered_ratios_df.head(6))

     country  year     ratio
0     Canada  2010  0.178301
1    Finland  2010  0.174349
2      Italy  2010  0.135654
3      Kenya  2010  0.003392
4     Norway  2010  0.330517
5  Singapore  2010  0.177087


In [ ]:
train_df_imputed = train_df.copy()
print(f"Missing values remaining: {train_df_imputed['num_sold'].isna().sum()}")

# Extract the year from the date
train_df_imputed['date'] = pd.to_datetime(train_df_imputed['date'])
train_df_imputed["year"] = train_df_imputed["date"].dt.year

# Loop through each year to perform imputation
for year in train_df_imputed["year"].unique():
    # Target ratio (Norway)
    target_ratio = gdp_per_capita_filtered_ratios_df.loc[
        (gdp_per_capita_filtered_ratios_df["year"] == year) &
        (gdp_per_capita_filtered_ratios_df["country"] == "Norway"), "ratio"
    ].values[0]

    # Impute Time Series 1: Canada, Discount Stickers, Holographic Goose
    current_ratio_can = gdp_per_capita_filtered_ratios_df.loc[
        (gdp_per_capita_filtered_ratios_df["year"] == year) &
        (gdp_per_capita_filtered_ratios_df["country"] == "Canada"), "ratio"
    ].values[0]
    ratio_can = current_ratio_can / target_ratio
    train_df_imputed.loc[
        (train_df_imputed["country"] == "Canada") &
        (train_df_imputed["store"] == "Discount Stickers") &
        (train_df_imputed["product"] == "Holographic Goose") &
        (train_df_imputed["year"] == year),
        "num_sold"
    ] = (
        train_df_imputed.loc[
            (train_df_imputed["country"] == "Norway") &
            (train_df_imputed["store"] == "Discount Stickers") &
            (train_df_imputed["product"] == "Holographic Goose") &
            (train_df_imputed["year"] == year),
            "num_sold"
        ] * ratio_can
    ).values

    # Impute Time Series 2-3: Canada, Premium Sticker Mart / Stickers for Less
    for store in ["Premium Sticker Mart", "Stickers for Less"]:
        current_ts = train_df_imputed.loc[
            (train_df_imputed["country"] == "Canada") &
            (train_df_imputed["store"] == store) &
            (train_df_imputed["product"] == "Holographic Goose") &
            (train_df_imputed["year"] == year)
        ]
        missing_ts_dates = current_ts.loc[current_ts["num_sold"].isna(), "date"]
        train_df_imputed.loc[
            (train_df_imputed["country"] == "Canada") &
            (train_df_imputed["store"] == store) &
            (train_df_imputed["product"] == "Holographic Goose") &
            (train_df_imputed["year"] == year) &
            (train_df_imputed["date"].isin(missing_ts_dates)),
            "num_sold"
        ] = (
            train_df_imputed.loc[
                (train_df_imputed["country"] == "Norway") &
                (train_df_imputed["store"] == store) &
                (train_df_imputed["product"] == "Holographic Goose") &
                (train_df_imputed["year"] == year) &
                (train_df_imputed["date"].isin(missing_ts_dates)),
                "num_sold"
            ] * ratio_can
        ).values

    # Impute Time Series 4: Kenya, Discount Stickers, Holographic Goose
    current_ratio_ken = gdp_per_capita_filtered_ratios_df.loc[
        (gdp_per_capita_filtered_ratios_df["year"] == year) &
        (gdp_per_capita_filtered_ratios_df["country"] == "Kenya"), "ratio"
    ].values[0]
    ratio_ken = current_ratio_ken / target_ratio
    train_df_imputed.loc[
        (train_df_imputed["country"] == "Kenya") &
        (train_df_imputed["store"] == "Discount Stickers") &
        (train_df_imputed["product"] == "Holographic Goose") &
        (train_df_imputed["year"] == year),
        "num_sold"
    ] = (
        train_df_imputed.loc[
            (train_df_imputed["country"] == "Norway") &
            (train_df_imputed["store"] == "Discount Stickers") &
            (train_df_imputed["product"] == "Holographic Goose") &
            (train_df_imputed["year"] == year),
            "num_sold"
        ] * ratio_ken
    ).values

    # Impute Time Series 5-6: Kenya, Premium Sticker Mart / Stickers for Less
    for store in ["Premium Sticker Mart", "Stickers for Less"]:
        current_ts = train_df_imputed.loc[
            (train_df_imputed["country"] == "Kenya") &
            (train_df_imputed["store"] == store) &
            (train_df_imputed["product"] == "Holographic Goose") &
            (train_df_imputed["year"] == year)
        ]
        missing_ts_dates = current_ts.loc[current_ts["num_sold"].isna(), "date"]
        train_df_imputed.loc[
            (train_df_imputed["country"] == "Kenya") &
            (train_df_imputed["store"] == store) &
            (train_df_imputed["product"] == "Holographic Goose") &
            (train_df_imputed["year"] == year) &
            (train_df_imputed["date"].isin(missing_ts_dates)),
            "num_sold"
        ] = (
            train_df_imputed.loc[
                (train_df_imputed["country"] == "Norway") &
                (train_df_imputed["store"] == store) &
                (train_df_imputed["product"] == "Holographic Goose") &
                (train_df_imputed["year"] == year) &
                (train_df_imputed["date"].isin(missing_ts_dates)),
                "num_sold"
            ] * ratio_ken
        ).values

    # Impute Time Series 7: Kenya, Discount Stickers, Kerneler
    current_ts = train_df_imputed.loc[
        (train_df_imputed["country"] == "Kenya") &
        (train_df_imputed["store"] == "Discount Stickers") &
        (train_df_imputed["product"] == "Kerneler") &
        (train_df_imputed["year"] == year)
    ]
    missing_ts_dates = current_ts.loc[current_ts["num_sold"].isna(), "date"]
    train_df_imputed.loc[
        (train_df_imputed["country"] == "Kenya") &
        (train_df_imputed["store"] == "Discount Stickers") &
        (train_df_imputed["product"] == "Kerneler") &
        (train_df_imputed["year"] == year) &
        (train_df_imputed["date"].isin(missing_ts_dates)),
        "num_sold"
    ] = (
        train_df_imputed.loc[
            (train_df_imputed["country"] == "Norway") &
            (train_df_imputed["store"] == "Discount Stickers") &
            (train_df_imputed["product"] == "Kerneler") &
            (train_df_imputed["year"] == year) &
            (train_df_imputed["date"].isin(missing_ts_dates)),
            "num_sold"
        ] * ratio_ken
    ).values

# Check for remaining missing values
print(f"Missing values remaining after imputation: {train_df_imputed['num_sold'].isna().sum()}")

# Manual imputation for specific IDs
train_df_imputed.loc[train_df_imputed["id"] == 23719, "num_sold"] = 4
train_df_imputed.loc[train_df_imputed["id"] == 207003, "num_sold"] = 195

# Final check for missing values
print(f"Final missing values remaining: {train_df_imputed['num_sold'].isna().sum()}")

Missing values remaining: 8871
Missing values remaining after imputation: 2
Final missing values remaining: 0


In [ ]:
def forecast_product_ratios(train_df, forecast_years):
    """
    Forecast product ratios for specific years based on historical data.

    """
    product_df = train_df.groupby(["date", "product"])["num_sold"].sum().reset_index()

    product_ratio_df = product_df.pivot(index="date", columns="product", values="num_sold")

    product_ratio_df = product_ratio_df.div(product_ratio_df.sum(axis=1), axis=0)
    product_ratio_df = product_ratio_df.stack().rename("ratios").reset_index()

    forecasted_ratios = []
    for base_year, target_year, year_shift in forecast_years:
        # Filter data for the base year
        forecast_df = product_ratio_df[product_ratio_df["date"].dt.year == base_year].copy()
        # Shift the date to the target year
        forecast_df["date"] += pd.DateOffset(years=year_shift)
        forecasted_ratios.append(forecast_df)

    forecasted_ratios_df = pd.concat(forecasted_ratios, ignore_index=True).drop_duplicates(subset=["date", "product"])

    return forecasted_ratios_df

In [ ]:
forecast_years = [(2015, 2017, 2), (2016, 2018, 2), (2015, 2019, 4)]
forecasted_ratios_df = forecast_product_ratios(train_df_imputed, forecast_years)
print("Forecasted Product Ratios (Sample):")
print(forecasted_ratios_df.head(5))

Forecasted Product Ratios (Sample):
        date             product    ratios
0 2017-01-01   Holographic Goose  0.053739
1 2017-01-01              Kaggle  0.350050
2 2017-01-01        Kaggle Tiers  0.263061
3 2017-01-01            Kerneler  0.155642
4 2017-01-01  Kerneler Dark Mode  0.177508


In [ ]:
original_train_df_imputed = train_df_imputed.copy()
train_df_imputed = train_df_imputed.groupby(["date"])["num_sold"].sum().reset_index()
train_df_imputed["year"] = train_df_imputed["date"].dt.year
train_df_imputed["month"] = train_df_imputed["date"].dt.month
train_df_imputed["day"] = train_df_imputed["date"].dt.day
train_df_imputed["day_of_week"] = train_df_imputed["date"].dt.dayofweek
train_df_imputed.head()

,date,num_sold,year,month,day,day_of_week
0,2010-01-01,85709.478063,2010,1,1,4
1,2010-01-02,82698.624480,2010,1,2,5
2,2010-01-03,88474.776409,2010,1,3,6
3,2010-01-04,68204.904090,2010,1,4,0
4,2010-01-05,65830.348330,2010,1,5,1


In [ ]:
df = train_df_imputed.copy()
df['iso_year'] = df['date'].dt.isocalendar().year
df['iso_week'] = df['date'].dt.isocalendar().week
df['week_id'] = df['iso_year'].astype(str) + '-W' + df['iso_week'].astype(str).str.zfill(2)
df['day_of_week'] = df['date'].dt.dayofweek

weekly_total = df.groupby('week_id')['num_sold'].sum().reset_index()
weekly_total.rename(columns={'num_sold': 'weekly_total_sold'}, inplace=True)
df = pd.merge(df, weekly_total, on='week_id')
df['daily_sales_ratio'] = df['num_sold'] / df['weekly_total_sold']
weekly_ratio = df.groupby(['week_id', 'day_of_week'])['daily_sales_ratio'].sum().reset_index()
weekly_ratio.head()

,week_id,day_of_week,daily_sales_ratio
0,2009-W53,4,0.333652
1,2009-W53,5,0.321931
2,2009-W53,6,0.344417
3,2010-W01,0,0.147759
4,2010-W01,1,0.142615


In [ ]:
test_df["date"] = pd.to_datetime(test_df["date"])

test_total_sales_df = test_df.groupby(["date"])["id"].first().reset_index().drop(columns="id")
test_total_sales_df["month"] = test_total_sales_df["date"].dt.month
test_total_sales_df["day"] = test_total_sales_df["date"].dt.day
test_total_sales_df["day_of_week"] = test_total_sales_df["date"].dt.dayofweek
test_total_sales_df.head()

,date,month,day,day_of_week
0,2017-01-01,1,1,6
1,2017-01-02,1,2,0
2,2017-01-03,1,3,1
3,2017-01-04,1,4,2
4,2017-01-05,1,5,3


In [ ]:
df = train_df_imputed.copy()
df['iso_year'] = df['date'].dt.isocalendar().year
df['iso_week'] = df['date'].dt.isocalendar().week
df['week_id'] = df['iso_year'].astype(str) + '-W' + df['iso_week'].astype(str).str.zfill(2)
df['day_of_week'] = df['date'].dt.dayofweek

weekly_total = df.groupby('week_id')['num_sold'].sum().reset_index()
weekly_total.rename(columns={'num_sold': 'weekly_total_sold'}, inplace=True)

df = pd.merge(df, weekly_total, on='week_id')
df['daily_sales_ratio'] = df['num_sold'] / df['weekly_total_sold']
weekly_ratio = df.groupby(['week_id', 'day_of_week'])['daily_sales_ratio'].sum().reset_index()
weekly_ratio['week_start'] = pd.to_datetime(weekly_ratio['week_id'] + '-1', format='%Y-W%W-%w')
first_monday = weekly_ratio['week_start'].min()
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [ ]:
day_of_week_ratio = (
    train_df_imputed.groupby("day_of_week")["num_sold"].mean() /
    train_df_imputed.groupby("day_of_week")["num_sold"].mean().mean()
).rename("day_of_week_ratios")

display(day_of_week_ratio)
train_df_imputed = pd.merge(train_df_imputed, day_of_week_ratio, how="left", on="day_of_week")
train_df_imputed["adjusted_num_sold"] = train_df_imputed["num_sold"] / train_df_imputed["day_of_week_ratios"]
difference_check = (train_df_imputed["num_sold"].sum() - train_df_imputed["adjusted_num_sold"].sum()) / train_df_imputed["num_sold"].sum()
print(f"The difference between original and adjusted total sales as a proportion is: {difference_check:.6f}")
print("\nAdjusted Sales (Adjusted num_sold):")
print(train_df_imputed[["date", "num_sold", "adjusted_num_sold"]].head())

,day_of_week_ratios
day_of_week,
0,0.943214
1,0.943346
2,0.943852
3,0.943740
4,1.000504
5,1.056339
6,1.169006


The difference between original and adjusted total sales as a proportion is: 0.000022

Adjusted Sales (Adjusted num_sold):
        date      num_sold  adjusted_num_sold
0 2010-01-01  85709.478063       85666.328441
1 2010-01-02  82698.624480       78287.939807
2 2010-01-03  88474.776409       75683.789324
3 2010-01-04  68204.904090       72311.183964
4 2010-01-05  65830.348330       69783.903150


In [ ]:
def prepare_test_data(train_df_imputed, test_total_sales_df, day_of_week_ratio):
    """
    Prepare the test data by calculating daily mean sales and incorporating day-of-week ratios.

    """
    train_last_x_years_df = train_df_imputed.loc[train_df_imputed["year"] >= 2010]
    train_day_mean_df = train_last_x_years_df.groupby(["month", "day"])["adjusted_num_sold"].mean().reset_index()
    test_total_sales_df = pd.merge(test_total_sales_df, train_day_mean_df, how="left", on=["month", "day"])
    test_total_sales_df = pd.merge(test_total_sales_df, day_of_week_ratio.reset_index(), how="left", on="day_of_week")
    test_total_sales_df["num_sold"] = test_total_sales_df["adjusted_num_sold"] * test_total_sales_df["day_of_week_ratios"]
    return test_total_sales_df

In [ ]:
def disaggregate_forecast(test_df, test_total_sales_df, store_weights, gdp_per_capita_filtered_ratios_df, forecasted_ratios_df):
    """
    Disaggregate total sales forecast by incorporating store, country, and product ratios.

    """
    # Add store ratios
    print(test_df.shape)
    store_weights_df = store_weights.reset_index()
    test_sub_df = pd.merge(test_df, test_total_sales_df, how="left", on="date")
    test_sub_df.rename(columns={"num_sold": "day_num_sold"}, inplace=True)
    print(f"After merge with test_total_sales_df: {test_sub_df.shape}")

    # Add product ratios
    test_sub_df = pd.merge(test_sub_df, store_weights_df, how="left", on="store")
    test_sub_df.rename(columns={"num_sold": "store_ratio"}, inplace=True)
    print(f"After merge with store_weights_df: {test_sub_df.shape}")

    # Add country ratios
    test_sub_df["year"] = test_sub_df["date"].dt.year
    test_sub_df = pd.merge(test_sub_df, gdp_per_capita_filtered_ratios_df, how="left", on=["year", "country"])
    test_sub_df.rename(columns={"ratio": "country_ratio"}, inplace=True)

    # Add product ratios
    test_sub_df = pd.merge(test_sub_df, forecasted_ratios_df, how="left", on=["date", "product"])
    test_sub_df.rename(columns={"ratios": "product_ratio"}, inplace=True)
    print(f"After merge with forecasted_ratios_df: {test_sub_df.shape}")

    # Adjust for bias for Kenya's GDP ratio
    test_sub_df.loc[test_sub_df['country'] == 'Kenya', 'country_ratio'] += 0.00249144564 * 1 / 10

    # Calculate final forecasted `num_sold`
    test_sub_df["num_sold"] = (
        test_sub_df["day_num_sold"] *
        test_sub_df["store_ratio"] *
        test_sub_df["country_ratio"] *
        test_sub_df["product_ratio"]
    )

    # Round `num_sold` to nearest integer
    test_sub_df["num_sold"] = test_sub_df["num_sold"].round()
    print(test_sub_df.shape)
    return test_sub_df

In [ ]:
test_total_sales_forecasted = prepare_test_data(train_df_imputed, test_total_sales_df, day_of_week_ratio)

# Disaggregate the forecasted data
test_forecast_disaggregated = disaggregate_forecast(
    test_df,
    test_total_sales_forecasted,
    store_weights,
    gdp_per_capita_filtered_ratios_df,
    forecasted_ratios_df
)

(98550, 5)
After merge with test_total_sales_df: (98550, 11)
After merge with store_weights_df: (98550, 12)
After merge with forecasted_ratios_df: (98550, 15)
(98550, 16)


In [ ]:
forecasted_ratios_df[forecasted_ratios_df[['date', 'product']].duplicated()]

,date,product,ratios


In [ ]:
submission = pd.DataFrame({"id": test_df["id"], "num_sold":test_forecast_disaggregated['num_sold']})

In [ ]:
submission.set_index("id", inplace=True)
submission["num_sold"] = submission["num_sold"].round()
submission.to_csv("submission_11.csv")

In [ ]:
test_forecast_disaggregated.id.nunique()

98550